# exploration how visa output differs from bank statements

In [1]:
import camelot
import pandas

In [2]:
def remove_whitespace(x):
    """remove all whitespaces from a given string"""
    if x is not None:
        x = x.split()
        x = ''.join(x)
    return x

vidi koja pravila data cleanupa ovdje trebaju

In [3]:
def extract_transactions(tables_transactions):
    """
    list of processed pdf pages -> dataframe with table that contains them all
    get the list of all monthly transactions, clean the data
    """
    transactions_list = []
    for table in tables_transactions:
        trans = table.df.values.tolist()
        for row in trans:
            transactions_list.append(row)
    transactions = pandas.DataFrame(transactions_list)

    transactions = transactions.applymap(remove_whitespace)

    transactions = transactions.drop(columns=[1])
    transactions.columns = ['date', 'trans_info', 'out']
   
    # cleanup data, remove all rows where date isn't date format
    transactions = transactions[transactions['date'].str.match('\d\d\.\d\d\.\d\d\d\d') == True]

    transactions.index = range(len(transactions.index))
    
    return transactions

figuring out column dimensions in the console
```
camelot -p 1 stream -plot text private_test_visa.pdf &

camelot -p 2 stream -plot text private_test_visa.pdf &

camelot -p 3 stream -plot text private_test_visa.pdf &

```

check in the tool where are the limits

final command

```
# table_areas = top_left x, y, bottom_right x, y // 0,0 is bottom left

camelot --format html --output test.html --pages 1 stream -T 0,470,550,180 -C 135,210,470 private_test_visa.pdf
camelot --format html --output test.html --pages 2 stream -T 0,715,550,0 -C 135,210,470 private_test_visa.pdf
camelot --format html --output test.html --pages 3 stream -T 0,715,550,0 -C 135,210,470 private_test_visa.pdf

```

pronadji boundaries
prva str je drugacija od ostalih

pronadji velicine kolona

begin balance je jednostavan, valjda
end balance je drugacije postavljen

In [4]:
def process_file(input_file):
    """ for a given path to a pdf file, extract the transactional data as a list of transactions (list)"""

    # table_areas = top_left x, y, bottom_right x, y // 0,0 is bottom left
    first_page = camelot.read_pdf(input_file, flavor='stream', pages='1',
                                          table_regions=['0,470,550,180'], columns=['135,210,470'])
    other_pages = camelot.read_pdf(input_file, flavor='stream', pages='2-end',
                                          table_regions=['0,715,550,0'], columns=['135,210,470'])
    all_pages = []
    all_pages += first_page
    all_pages += other_pages
    
    result = extract_transactions(all_pages)
    return result #.values.tolist()   